# Décorateurs en Python : Fonctions d’Ordre Supérieur et Plus

Dans cette section, nous allons explorer les **décorateurs**, une fonctionnalité puissante de Python basée sur les **fonctions d’ordre supérieur**. Nous aborderons leur création, leur utilisation pour modifier ou étendre le comportement des fonctions.

## Qu’est-ce qu’un Décorateur ?
Un décorateur est une fonction qui prend une autre fonction en entrée, ajoute ou modifie son comportement, et renvoie une nouvelle fonction. C’est une application directe des **fonctions d’ordre supérieur**, où les fonctions sont traitées comme des objets manipulables.

Commençons par les bases !

## Fonctions d’Ordre Supérieur

Une **fonction d’ordre supérieur** est une fonction qui :
- Prend une ou plusieurs fonctions comme arguments.
- Ou renvoie une fonction comme résultat.

### Exemple Simple
Illustrons ce concept avant de passer aux décorateurs.

In [2]:
def appliquer_operation(fonction, valeur: int) -> int:
    """Applique une fonction donnée à une valeur."""
    return fonction(valeur)

In [3]:

def doubler(x: int) -> int:
    return x * 2


def tripler(x: int) -> int:
    return x * 3


In [4]:
print(appliquer_operation(doubler, 5)) 


10


In [5]:
print(appliquer_operation(tripler, 5))  

15



- **`appliquer_operation`** : Prend une fonction (`doubler` ou `tripler`) comme argument et l’applique à `valeur`.
- **Flexibilité** : Une seule fonction peut utiliser différentes logiques.

Ceci est la base des décorateurs. Passons à leur création !

## Création d’un Décorateur

Un décorateur est une fonction qui enveloppe une autre fonction pour ajouter un comportement avant, après ou autour de son exécution.

### Syntaxe
```python
def decorateur(fonction):
    def enveloppe(*args, **kwargs):
        # Avant
        resultat = fonction(*args, **kwargs)
        # Après
        return resultat
    return enveloppe

@decorateur
def ma_fonction():
    pass
```

### Exemple Simple

Ajoutons un message avant et après une fonction.

In [6]:
def log_execution(fonction):
    """Affiche un message avant et après l’exécution de la fonction."""
    def enveloppe():
        print(f"Début de {fonction.__name__}")
        fonction()
        print(f"Fin de {fonction.__name__}")
    return enveloppe



In [7]:

@log_execution
def dire_bonjour():
    """Dit bonjour."""
    print("Bonjour !")


In [8]:
dire_bonjour()

Début de dire_bonjour
Bonjour !
Fin de dire_bonjour


## Décorateurs avec Arguments

Pour gérer des fonctions avec paramètres, utilisez `*args` et `**kwargs`.

### Exemple
Mesurons le temps d’exécution d’une fonction.

In [9]:
import time


def mesurer_temps(fonction):
    """Mesure le temps d’exécution d’une fonction."""
    def enveloppe(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        print(f"{fonction.__name__} a pris {fin - debut:.4f} secondes")
        return resultat
    return enveloppe


@mesurer_temps
def calculer_somme(n: int) -> int:
    """Calcule la somme des nombres jusqu’à n."""
    return sum(range(n))



In [10]:
print(calculer_somme(1000000))

calculer_somme a pris 0.0089 secondes
499999500000


## Décorateurs avec Paramètres

Pour passer des paramètres au décorateur lui-même, ajoutez une couche de fonction supplémentaire.

### Syntaxe
```python
def decorateur_parametrable(param):
    def decorateur(fonction):
        def enveloppe(*args, **kwargs):
            # Utilise param
            return fonction(*args, **kwargs)
        return enveloppe
    return decorateur
```

### Exemple
Créons un décorateur pour répéter une fonction.

In [11]:
def repeter(nb_fois: int):
    """Répète l’exécution d’une fonction un nombre donné de fois."""
    def decorateur(fonction):
        def enveloppe(*args, **kwargs):
            for _ in range(nb_fois):
                fonction(*args, **kwargs)
        return enveloppe
    return decorateur


@repeter(3)
def saluer(nom: str):
    """Salue une personne."""
    print(f"Salut, {nom} !")


In [12]:
saluer("Alice")

Salut, Alice !
Salut, Alice !
Salut, Alice !


## Préserver les Métadonnées avec `wraps`

Les décorateurs remplacent la fonction originale par `enveloppe`, perdant ses métadonnées (nom, docstring). Utilisez `functools.wraps` pour les conserver.

### Exemple
Comparons sans et avec `wraps`.

In [13]:
from functools import wraps


def sans_wraps(fonction):
    def enveloppe():
        return fonction()
    return enveloppe


def avec_wraps(fonction):
    @wraps(fonction)
    def enveloppe():
        return fonction()
    return enveloppe



In [14]:

@sans_wraps
def test1():
    """Fonction test 1."""
    print("Test 1")


@avec_wraps
def test2():
    """Fonction test 2."""
    print("Test 2")


In [15]:
print(test1.__name__, test1.__doc__)  


enveloppe None


In [16]:
print(test2.__name__, test2.__doc__)  

test2 Fonction test 2.


## Cas d’Usage Courants

### 1. Journalisation
Ajoute des logs à une fonction.

### 2. Validation
Vérifie les arguments avant exécution.

### Exemple de Journalisation

In [17]:
import logging
from functools import wraps

logging.basicConfig(level=logging.INFO)


def journaliser(fonction):
    """Ajoute des logs avant et après l’exécution."""
    @wraps(fonction)
    def enveloppe(*args, **kwargs):
        logging.info(f"Appel de {fonction.__name__} avec {args}, {kwargs}")
        resultat = fonction(*args, **kwargs)
        logging.info(f"{fonction.__name__} a retourné {resultat}")
        return resultat
    return enveloppe


@journaliser
def multiplier(a: int, b: int) -> int:
    """Multiplie deux nombres."""
    return a * b


In [18]:

print(multiplier(3, 4))

INFO:root:Appel de multiplier avec (3, 4), {}
INFO:root:multiplier a retourné 12


12


### Exemple de Validation
Vérifions que les arguments sont positifs.

In [19]:
from functools import wraps


def positifs(fonction):
    """Vérifie que tous les arguments numériques sont positifs."""
    @wraps(fonction)
    def enveloppe(*args, **kwargs):
        for arg in args:
            if isinstance(arg, (int, float)) and arg < 0:
                raise ValueError(f"Argument négatif détecté : {arg}")
        for valeur in kwargs.values():
            if isinstance(valeur, (int, float)) and valeur < 0:
                raise ValueError(f"Argument négatif détecté : {valeur}")
        return fonction(*args, **kwargs)
    return enveloppe


@positifs
def additionner(a: int, b: int) -> int:
    """Additionne deux nombres."""
    return a + b


In [20]:

print(additionner(2, 3)) 


5


In [21]:

try:
    additionner(-1, 2)
except ValueError as e:
    print(f"Erreur : {e}") 

Erreur : Argument négatif détecté : -1


## Exemple Avancé : Cache Mémoïsé

Créons un décorateur qui met en cache les résultats d’une fonction (mémoïsation).

In [22]:
from functools import wraps


def memoize(fonction):
    """Met en cache les résultats d’une fonction pour éviter les recalculs."""
    cache = {}
    
    @wraps(fonction)
    def enveloppe(*args):
        if args not in cache:
            cache[args] = fonction(*args)
            print(f"Calcul pour {args}, mis en cache")
        else:
            print(f"Utilisation du cache pour {args}")
        return cache[args]
    return enveloppe


@memoize
def fibonacci(n: int) -> int:
    """Calcule le n-ième nombre de Fibonacci."""
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)


In [23]:

print(fibonacci(5))


Calcul pour (1,), mis en cache
Calcul pour (0,), mis en cache
Calcul pour (2,), mis en cache
Utilisation du cache pour (1,)
Calcul pour (3,), mis en cache
Utilisation du cache pour (2,)
Calcul pour (4,), mis en cache
Utilisation du cache pour (3,)
Calcul pour (5,), mis en cache
5


In [24]:
print(fibonacci(5))

Utilisation du cache pour (5,)
5


## Conclusion

Cette section vous a permis de maîtriser :
- Les **fonctions d’ordre supérieur** comme base des décorateurs.
- La **création et utilisation de décorateurs** pour ajouter des comportements (logs, validation, temps).
- Des **cas d’usage courants** comme la mémoïsation et la journalisation.

Les décorateurs sont un outil élégant pour étendre vos fonctions sans les modifier directement. Expérimentez avec ces exemples pour créer vos propres décorateurs adaptés à vos besoins !